In [1]:
import pandas as pd
import json
import numpy as np
from glob import glob
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib

matplotlib.rc('pdf', fonttype=42)
matplotlib.rc('ps', fonttype=42)

import os
sns.set(font_scale=1.25, style='whitegrid')
sns.set(palette='magma')

# rdir = '../results/'
rdir = '../results_blackbox/'
# figdir = 'figs/black-box' 
figdir = '../paper/'
# Set to path to PMLB datasets
datadir = '../datasets/pmlb/datasets/'
print('figdir:',figdir)
def save(name='tmp',h=None):
    name = name.strip().replace(' ','-').replace('%','pct')
    if h == None:
        h = plt.gcf()
    h.tight_layout()
    print('saving',figdir+'/'+name+'.pdf')
    if not os.path.exists(figdir):
        os.makedirs(figdir)
    plt.savefig(figdir+'/'+name+'.pdf', bbox_inches='tight')
    
symbolic_algs = [
    'AFP', 
    'AFP_FE',
    'BSR',
    'DSR',
    'FFX',

    # brush with mabs
    'Baseline',
    'B-wo-split',
    "C-D-UCB1",
    "C-D-TS",
    "D-UCB1",
    "D-TS",

    'FEAT S-Split',
    'FEAT SD-Split',
    'FEAT D-Split',
    'FEAT',

    'EPLEX',
    'GP-GOMEA',
    'gplearn',
    'ITEA', 
    'MRGP', 
    'Operon',
    'SBP-GP',
    'AIFeynman'
]
nongp_algs = [
    'BSR',
    'DSR',
    'AIFeynman',
    "E2E"
]
gp_algs = [
    'AFP', 
    'AFP_FE',
    'FFX',

    'FEAT S-Split',
    'FEAT SD-Split',
    'FEAT D-Split',
    'FEAT',

    # brush with mabs
    'Baseline',
    'Baseline-500',
    'B-wo-split',
    "C-D-UCB1",
    "C-D-TS",
    "D-UCB1",
    "D-TS",

    'EPLEX',
    'GP-GOMEA',
    'gplearn',
    'ITEA', 
    'MRGP', 
    'Operon',
    'SBP-GP',
]

figdir: ../paper/


In [2]:
df_results = pd.read_feather('../results/black-box_results_local.feather')

df_results

,dataset,algorithm,random_state,training time (s),symbolic_model,mse_test,mae_test,r2_test,model_size,params_str,training time (hr),r2_zero_test,friedman_dataset,symbolic_alg
0,195_auto_price,Baseline,4426,406.187133,"If(highway-mpg>-2.16,If(city-mpg>-1.77,Add(0.7...",1.345981e+07,2338.477209,0.606140,28,"{'pop_size': 1000, 'max_gen': 250, 'verbosity'...",0.112830,0.606140,False,True
1,195_auto_price,E2E,29910,36.796347,(-1.21 + (0.06420000000000001 * abs((((-0.508 ...,1.345474e+07,2674.015212,0.201093,142,"{'max_input_points': 200, 'max_number_bags': -...",0.010221,0.201093,False,False
2,195_auto_price,C-D-UCB1,23654,1192.566812,"If(highway-mpg>-2.05,0.89*curb-weight,4.18)",6.603523e+06,1821.365112,0.834649,8,"{'pop_size': 1000, 'max_gen': 250, 'verbosity'...",0.331269,0.834649,False,True
3,195_auto_price,E2E,21962,29.160861,(((5.89 + (-7.799549988122946e-16 + (0.9999999...,1.510116e+07,2639.802957,0.628192,175,"{'max_input_points': 200, 'max_number_bags': -...",0.008100,0.628192,False,False
4,195_auto_price,uDSR,14423,266.004804,x8 + sin(x7 - x8),5.639009e+06,1729.632537,0.752779,6,"{'config': {'experiment': {'logdir': None, 'ex...",0.073890,0.752779,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10457,1030_ERA,Baseline-500,5390,454.929800,"Sub(Add(Tan(Sub(0.30*Add(in1,in2),-0.40)),-0.0...",2.503545e+00,1.281969,0.380517,19,"{'pop_size': 1000, 'max_gen': 500, 'verbosity'...",0.126369,0.380517,False,True
10458,1030_ERA,B-wo-split,16850,306.386404,"Add(0.40*Add(Add(in2,Tanh(in3)),in1),0.14*in4)",2.433807e+00,1.217885,0.259391,12,"{'pop_size': 1000, 'max_gen': 500, 'verbosity'...",0.085107,0.259391,False,True
10459,1030_ERA,TPSR+E2E,16850,510.359567,(0.15712366321691942 - 0.04964984260383692*cos...,3.362401e+00,1.530400,-0.023181,43,{'random_state': 16850},0.141767,0.000000,False,False
10460,1030_ERA,D-UCB1,21962,210.635783,"Sinh(6.63*Add(Sin(Sub(Sub(0.14*in1,-0.13*in2),...",2.452704e+00,1.236894,0.321591,30,"{'pop_size': 1000, 'max_gen': 250, 'verbosity'...",0.058510,0.321591,False,True


In [3]:
df_results.columns

Index(['dataset', 'algorithm', 'random_state', 'training time (s)',
       'symbolic_model', 'mse_test', 'mae_test', 'r2_test', 'model_size',
       'params_str', 'training time (hr)', 'r2_zero_test', 'friedman_dataset',
       'symbolic_alg'],
      dtype='object')

In [5]:
for brush in ['Baseline',
    'Baseline-500',
    'B-wo-split',
    "C-D-UCB1",
    "C-D-TS",
    "D-UCB1",
    "D-TS",]:
    df_aux = df_results[
        (df_results['algorithm']==brush)
        & (df_results['symbolic_model'].str.contains("If"))
    ]
    print(brush, df_aux.shape[0])

Baseline 153
Baseline-500 40
B-wo-split 0
C-D-UCB1 184
C-D-TS 125
D-UCB1 166
D-TS 203


In [14]:
with pd.option_context('display.max_colwidth', None):
    df_aux = df_results[
        (df_results['algorithm']=='Baseline-500')
        & (df_results['symbolic_model'].str.contains("If"))
    ][['dataset', 'r2_test', 'mse_test', 'symbolic_model']]

    df_aux.to_csv("./inspecting_models.csv")
    display(df_aux)

,dataset,r2_test,mse_test,symbolic_model
136,nikuradse_1,0.989118,3.065822e-04,"Mul(Atan(0.67*Sub(Sub(-0.56,Exp(-1.22*log_Re)),Div(Sinh(3.05*r_k),Sub(1.29,-0.63*log_Re)))),Add(Logistic(Add(Sub(2.88,-2.91*log_Re),-0.22*r_k)),If(r_k>-0.72,0.07,Atan(0.80))))"
145,nikuradse_1,0.994113,1.429491e-04,"Sub(0.66*Sub(1.38*Log1p(0.24*Exp(-2.34*Log1p(r_k))),Tanh(Add(0.66*log_Re,1.36))),Logistic(1.01*Mul(Add(Log1p(1.16*r_k),-0.55*log_Re),Add(2.49*log_Re,If(r_k>-0.75,1.96,3.10)))))"
157,nikuradse_1,0.976120,6.448645e-04,"If(r_k>-0.71,Sub(0.77*Exp(-1.33*Sinh(Sub(r_k,0.13*log_Re))),Log(Add(4.29,log_Re))),1.48)"
158,nikuradse_1,0.987709,2.780401e-04,"Mul(Log1p(0.01*Sub(0.34*Add(4.77*Ceil(5.70*r_k),Tanh(2.69*log_Re)),Atan(Add(4.75,3.02*log_Re)))),Add(20.88*Logabs(Sinh(-1.02*Logabs(0.84*r_k))),-2.47*Tanh(If(r_k>-0.73,Add(1.97,3.21*log_Re),Pow(1020.90,0.14*log_Re)))))"
1548,678_visualizing_environmental,-0.130632,1.691622e+01,"Sub(Add(Div(0.04*ozone,Sub(temperature,Sub(0.55,0.09))),-0.54*ozone),-0.14*Tan(Mul(Div(Cos(1.66*temperature),0.84),Add(radiation,If(temperature>-2.07,0.40,-17.22)))))"
1817,611_fri_c3_100_5,0.950705,5.892041e-02,"Add(0.88*Add(If(oz3>2.52,0.10*oz1,0.08*Mul(Add(-1.37*oz2,-0.34),Ceil(oz2))),Abs(Sub(Sqrtabs(oz3),Sub(0.56,-0.05*oz5)))),0.71*Add(Sin(Sub(Mul(oz2,-1.03*oz1),1.84)),1.01*Sub(1.52*Sin(1.07*Sub(-1.53*oz2,oz1)),2.84*Log1p(-0.19*oz4))))"
1871,611_fri_c3_100_5,0.897915,4.822517e-02,"Add(Mul(Sin(Add(1.48*oz2,Add(oz1,0.49))),Sub(-2.12*Sub(Logistic(oz2),Square(0.25*oz5)),0.13)),Add(0.46*Atan(oz4),If(oz5>2.80,1.48,Add(Cosh(0.51*oz3),-1.09))))"
1883,611_fri_c3_100_5,0.963047,3.416104e-02,"Mul(Add(2.64*Sin(Sub(-1.44*oz2,Sub(oz1,-0.35))),Add(oz4,If(oz3>2.47,1.29,0.09))),0.81*Logistic(0.81*oz2))"
2068,561_cpu,0.889203,3.154043e+03,"Add(Sub(Add(Atan(Sub(0.02*vendor,0.07*MMAX)),-1.24),-0.09*CACH),Mul(Div(Atan(Exp(0.19*MMIN)),Add(If(MMIN>5.25,0.11,-0.02*CHMIN),Logistic(0.04*vendor))),0.25*Exp(Add(0.34*MMAX,Exp(0.06*CACH)))))"
2166,644_fri_c4_250_25,0.947333,3.759526e-02,"0.06*Sub(20.19*Sin(Mul(Add(Add(-2.12*oz2,-1.41*oz1),1.14),Log1p(Exp(0.30*oz1)))),Sub(Add(9.71*Tanh(Sub(oz3,-0.76)),Add(If(oz5>3.17,-4.90*oz4,-6.59*oz4),-4.39*oz5)),Square(oz5)))"


In [ ]:
Add(1.00*Add(Sub(5.98*average_salary,If(average_salary>1.46,0.88*Log1p(2.09),If(average_salary>1.23,0.80*average_salary,1.00))),-3.49*full_prof_salary),Sub(-1.73*assoc_prof_salary,Sub(Div(-0.00*assoc_prof_salary,Add(full_prof_salary,0.57)),Exp(Div(-0.00*CIC_institutions,2.31*average_salary)))))